# About: ロギングドライバを変更する--fluentd
---

`compose.yaml`を編集してMoodleコンテナのロギングドライバを変更します。

## 概要

VCノードとして起動したノードではfluentdがサービスとして実行されています。このNotebookではコンテナログやコンテナ内で実行されているサービスのログをVCノードのfluentdを経由して他ホストのfluentdに転送する設定を行います。

![処理の流れ](images/moodle-052-01.png)

1. 他ホストのfluentdにログを転送するfluentdの設定ファイルを配置する
1. `/var/log/httpd/*` などホスト側に出力されるログファイルをfluentdで読み込む設定ファイルを配置する
1. ホスト環境の `fluentd` を再起動して追加した設定を反映する
1. コンテナのロギングドライバ設定を追加した`compose.yaml`を配置する
1. ロギング設定の変更を反映するためにコンテナを起動し直す

### 前提条件

このNotebookでの設定手順の前提となる条件を以下に示します。

* 転送先の fluentd が、Moodle構築環境から転送したログを受け取ることができること
* 「011-VCノードの作成.ipynb」でノードを作成していること

このNotebookで示している設定手順はVCノードにあらかじめ組み込まれているfluentdの機能を前提としたものとなっています。そのためノードの作成は「011-VCノードの作成.ipynb」で行なう必要があります。

## 準備

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切であることをチェックします。またfluentdの設定を行うための前提条件に合致しているかの簡単なチェックも行います。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

構築環境でfluentdサービスが実行されていることを確認します。

In [ ]:
!ansible {target_group} -b -a 'systemctl status fluentd'

ポート 24224 でリッスンしていることを確認します。

In [ ]:
!ansible {target_group} -m shell -a '/sbin/ss -nlt | grep 24224'

## パラメータ設定

### 転送先となるfluentdの指定

ログの送信先となる fluentd サーバを指定します。

> VCノードではログをVCコントローラに転送する設定があらかじめ組み込まれています。VCコントローラのみにログを転送すれば十分な場合はこの節での設定は不要です。次節の「3.2 転送対象とするホスト環境のログを指定する」にスキップしてください。

ログの転送先となるfluentdサーバのホスト名（IPアドレス）を指定してください。

In [ ]:
# (例)
# fluentd_server = 'fluentd.example.org'
# fluentd_server = '192.168.200.200'

fluentd_server = 

ログの転送先となるfluentdサーバのポート番号を指定してください。

In [ ]:
# (例)
# fluentd_port = 24224

fluentd_port = 24224

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

パラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    logging_driver='fluentd',
    fluentd_server=fluentd_server,
    fluentd_port=fluentd_port,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

### 転送対象とするホスト環境のログを指定する

コンテナで実行しているApache HTTP serverなどのサービスのログはホスト環境の`/var/log/`に記録されています。これらのログを fluentdから読み込むようにする際に必要となるパラメータの指定を行います。

ホスト側に記録されるログとコンテナとの関係を次表に示します。

|コンテナ|パス|
|:---|:---|
|db|/var/log/mysql/error.log|
|moodle|/var/log/httpd/access_log|
|moodle|/var/log/httpd/error_log|
|httpd|/var/log/httpd/access_log|
|httpd|/var/log/httpd/error_log|
|httpd|/var/log/httpd/ssl_access_log|
|httpd|/var/log/httpd/ssl_error_log|
|shibboleth|/var/log/httpd/access_log|
|shibboleth|/var/log/httpd/error_log|
|shibboleth|/var/log/httpd/ssl_access_log|
|shibboleth|/var/log/httpd/ssl_error_log|
|shibboleth|/var/log/shibboleth/shibd.log|
|shibboleth|/var/log/shibboleth/shibd_warn.log|
|shibboleth|/var/log/shibboleth/signature.log|
|shibboleth|/var/log/shibboleth/transaction.log|


これらのログは[in_tail](https://docs.fluentd.org/input/tail)プラグインの機能を利用してMoodle構築環境で実行しているfluentdから読みむように設定します。in_tailプラグインに関するパラメータを次のセルで指定してください。例えば、
```python
[
    {
        "path": "/var/log/httpd/access_log",
        "tag": "ocs.app.httpd",
        "pos_file": "/var/run/log/fluentd/httpd_access_log.pos",
        "parse": {
            "@type": "apache2",
        },
    },
]
```
のように指定すると、以下に示すfluentdの設定が生成されます。

```
<source>
    @type tail
    path "/var/log/httpd/access_log"
    tag "ocs.app.httpd"
    pos_file "/var/run/log/fluentd/httpd_access_log.pos"
    <parse>
        @type "apache2"
    </parse>
</source>
```


> httpdコンテナ、shibbolethコンテナに関連するログファイルはコメントアウトしています。必要に応じてコメントを外してください。

In [ ]:
fluentd_configs = [
    {
        "path": "/var/log/mysql/error.log",
        "tag": "ocs.app.mariadb",
        "parse": {
            "@type": "none",
        },
        "pos_file": "/var/run/log/fluentd/mysql_error_log.pos",
    },
    {
        "path": "/var/log/httpd/access_log",
        "tag": "ocs.app.httpd",
        "parse": {
            "@type": "apache2",
        },
        "pos_file": "/var/run/log/fluentd/httpd_access_log.pos",
    },
    {
        "path": "/var/log/httpd/error_log",
        "tag": "ocs.app.httpd",
        "parse": {
            "@type": "apache_error",
        },
        "pos_file": "/var/run/log/fluentd/httpd_error_log.pos",
    },

### httpd
#    {
#        "path": "/var/log/httpd/ssl_access_log",
#        "tag": "ocs.app.httpd",
#        "parse": {
#            "@type": "apache2",
#        },
#        "pos_file": "/var/run/log/fluentd/httpd_ssl_access_log.pos",
#    },
#    {
#        "path": "/var/log/httpd/ssl_error_log",
#        "tag": "ocs.app.httpd",
#        "parse": {
#            "@type": "apache_error",
#        },
#        "pos_file": "/var/run/log/fluentd/httpd_ssl_error_log.pos",
#    },
    
### shibboleth
#    {
#        "path": "/var/log/shibboleth/shibd.log",
#        "tag": "ocs.app.shibboleth",
#        "parse": {
#            "@type": "none",
#        },
#        "pos_file": "/var/run/log/fluentd/shibboleth_shibd_log.pos",
#    },
#    {
#        "path": "/var/log/shibboleth/shibd_warn.log",
#        "tag": "ocs.app.shibboleth.warn",
#        "parse": {
#            "@type": "none",
#        },
#        "pos_file": "/var/run/log/fluentd/shibboleth_shibd_warn_log.pos",
#    },
#    {
#        "path": "/var/log/shibboleth/transaction.log",
#        "tag": "ocs.app.shibboleth.transaction",
#        "parse": {
#            "@type": "none",
#        },
#        "pos_file": "/var/run/log/fluentd/shibboleth_transaction_log.pos",
#    },
#    {
#        "path": "/var/log/shibboleth/signature.log",
#        "tag": "ocs.app.shibboleth.signature",
#        "parse": {
#            "@type": "none",
#        },
#        "pos_file": "/var/run/log/fluentd/shibboleth_signature_log.pos",
#    },
]

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    logging_driver='fluentd',
    fluentd_configs=fluentd_configs,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

### コンテナのログ設定に関するパラメータの指定

コンテナのロギング設定に関するオプションを指定します。指定するパラメータは[Fluentd logging driver - Options](https://docs.docker.com/config/containers/logging/fluentd/#options)に記されているものをキー、バリューの形で指定します。

例えば `fluentd-buffer-limit` に `2097152` を設定する場合は、以下のような指定を行います。

```python
logging_options = {
    "fluentd-buffer-limit": 2097152,
}
```

コンテナログは構築環境のVCノードで実行しているfluentdを経由して送信されるため`fluentd-address`の値を`logging_options`で指定する必要はありません。

In [ ]:
logging_options = {

}

ロギングオプションの`tag`に指定するパラメータを指定します。指定する値はコンテナごとに異なる値を指定します。次のセルではキーに`compose.yaml`のサービス名を、バリューに`tag`に設定する値をdict 型の変数として指定を行います。

「021-Moodleコンテナの起動.ipynb」で構築した環境では`db`コンテナ、`moodle`コンテナの2つのコンテナ飲みで構成されています。「071-HTTPS接続に変更する.ipynb」「221-Shibboleth認証の設定.ipynb」を実行しコンテナ構成が変更された場合を考慮して`httpd`コンテナ、`shibboleth`コンテナに対するタグの値もあらかじめ、ここで指定することを推奨します。

In [ ]:
logging_tags = {
    'db': 'db',
    'moodle': 'moodle',
    'httpd': 'httpd',
    'shibboleth': 'shibboleth',
}

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    target_group,
    _file="50-logging.yml",
    logging_driver='fluentd',
    logging_options=logging_options if 'logging_options' in vars() else {},
    logging_tags=logging_tags if 'logging_tags' in vars() else {},
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/50-logging.yml

## fluentdの設定を変更する



### 設定ファイルを配置する

fluentdの設定ファイルを構築環境に配置します。

#### 転送先に関する設定ファイルの配置

実際に設定ファイルを配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -b -CDv -m template \
    -a 'src=template/fluentd/forward.conf.j2 \
    dest=/etc/fluentd/config.d/ocs-output/forward.conf backup=yes'

設定ファイルを構築環境に配置します。

In [ ]:
!ansible {target_group} -b -m template \
    -a 'src=template/fluentd/forward.conf.j2 \
    dest=/etc/fluentd/config.d/ocs-output/forward.conf backup=yes'

#### ログの読み取り先を指定する設定ファイルの配置

実際に設定ファイルを配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -b -CDv -m template \
    -a 'src=template/fluentd/moodle.conf.j2 \
    dest=/etc/fluentd/config.d/moodle.conf backup=yes'

設定ファイルを構築環境に配置します。

In [ ]:
!ansible {target_group} -b -m template \
    -a 'src=template/fluentd/moodle.conf.j2 \
    dest=/etc/fluentd/config.d/moodle.conf backup=yes'

### 変更を反映する

fluentdサービスの再起動を行い、変更を反映させます。

fluentdサービスの再起動を行います。

In [ ]:
!ansible {target_group} -b -a 'systemctl restart fluentd'

fluentdサービスの状態を確認します。エラーになっていないことを確認してください。

In [ ]:
!ansible {target_group} -b -a 'systemctl status fluentd'

## コンテナのロギング設定を変更する

コンテナのログをfluentdに転送するための構成変更を行います。

### compose.yaml の配置

ロギングドライバの設定を追加した`compose.yaml`を構築環境に配置します。

実際に`compose.yaml`を配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/compose.yaml.j2 \
    dest=/srv/moodle/compose.yaml backup=yes'

`compose.yaml`を構築環境に配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/compose.yaml.j2 \
    dest=/srv/moodle/compose.yaml backup=yes'

### コンテナ設定の変更を反映する

コンテナ設定の変更反映をするためにコンテナを再起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up --wait --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

送信先のfluentdにログが転送されていることを確認してください。